In [2]:
import sklearn.metrics as skm
import sys
import numpy as np
from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay
import matplotlib.pyplot as plt

ssl_features_train = np.load("./old_but_full_features/self_supervised_cifar100/features_train2.npy")
ssl_labels_train = np.load("./old_but_full_features/self_supervised_cifar100/labels_train2.npy")
ssl_features_test = np.load("./old_but_full_features/self_supervised_cifar100/features_test2.npy")
ssl_features_ood = np.load("./old_but_full_features/self_supervised_cifar100/features_ood2.npy")


sp_features_train = np.load("./old_but_full_features/supervised_cifar100/features_train2.npy")
sp_labels_train = np.load("./old_but_full_features/supervised_cifar100/labels_train2.npy")
sp_features_test = np.load("./old_but_full_features/supervised_cifar100/features_test2.npy")
sp_features_ood = np.load("./old_but_full_features/supervised_cifar100/features_ood2.npy")


In [78]:
import matplotlib.pyplot as plt

def dnorm(ftrain, ftest, food):
    
    print("ftrain size is:", ftrain.shape)
    ftrain /= np.linalg.norm(ftrain, axis=-1, keepdims=True) + 1e-10
    ftest /= np.linalg.norm(ftest, axis=-1, keepdims=True) + 1e-10
    food /= np.linalg.norm(food, axis=-1, keepdims=True) + 1e-10

    m, s = np.mean(ftrain, axis=0, keepdims=True), np.std(ftrain, axis=0, keepdims=True)
    print("mean value is:", np.mean(m))
    print("std value is:", np.mean(s))
#     m2, s2 = np.mean(ftest, axis=0, keepdims=True), np.std(ftest, axis=0, keepdims=True)
#     m3, s3 = np.mean(food, axis=0, keepdims=True), np.std(food, axis=0, keepdims=True)

    ftrain = (ftrain - m) / (s + 1e-10)
    ftest = (ftest - m) / (s + 1e-10)
    food = (food - m) / (s + 1e-10)
#     print("ftrain shape is:", ftrain.shape)
#     ftrain = (ftrain - m) / (s + 1e-10)
#     ftest = (ftrain - m2) / (s2 + 1e-10)
#     food = (ftrain - m3) / (s3 + 1e-10)

#     plt.figure(0).clf()
#     plt.plot(food[960])
#     plt.hist(ftrain[60], density=True, bins=100)
#     plt.show()
    
    
    return ftrain, ftest, food

ssl_features_train,  ssl_features_test, ssl_features_ood= dnorm(ssl_features_train, ssl_features_test, ssl_features_ood)
sp_features_train,  sp_features_test, sp_features_ood= dnorm(sp_features_train, sp_features_test, sp_features_ood)

alpha = 0.5
# fl_features_train = alpha * ssl_features_train + (1-alpha) * sp_features_train
# fl_features_test = alpha * ssl_features_test + (1-alpha) * sp_features_test
# fl_features_ood = alpha * ssl_features_ood + (1-alpha) * sp_features_ood
fl_features_train = np.concatenate((ssl_features_train, sp_features_train), axis = 1)
fl_features_test = np.concatenate((ssl_features_test, sp_features_test), axis = 1)
fl_features_ood = np.concatenate((ssl_features_ood, sp_features_ood), axis = 1)
print("new fl_feature_train size is:", fl_features_train.shape)

ftrain size is: (50000, 2048)
mean value is: 2.583538e-09
std value is: 0.022097033
ftrain size is: (50000, 2048)
mean value is: -7.845433e-07
std value is: 0.022097047
new fl_feature_train size is: (50000, 4096)


In [158]:

i, j = 5, 6

In [159]:
# calcualte cosine similarity of ssl, sl, and ensemble 
from scipy.spatial import distance

# reshape(1, -1)

xc = [ssl_features_train[ssl_labels_train == i] for i in np.unique(ssl_labels_train)]

print(xc[0].shape)

all_embedding = []
for x in xc: 
    
    cla_embedding = np.mean(x, axis=0, keepdims=True)
#     print(cla_embedding.shape)
    all_embedding.append(cla_embedding)
all_embedding = np.array(all_embedding)

print(all_embedding.shape)


Inter_class_dispersion = 1 - distance.cosine(all_embedding[i], all_embedding[j])

print(180 * np.arccos(Inter_class_dispersion)/math.pi)













(5000, 2048)
(10, 1, 2048)
97.2438270342927


In [160]:
xc = [sp_features_train[sp_labels_train == i] for i in np.unique(sp_labels_train)]

print(xc[0].shape)

all_embedding = []
for x in xc: 
    
    cla_embedding = np.mean(x, axis=0, keepdims=True)
#     print(cla_embedding.shape)
    all_embedding.append(cla_embedding)
all_embedding = np.array(all_embedding)

print(all_embedding.shape)


Inter_class_dispersion = 1 - distance.cosine(all_embedding[i], all_embedding[j])

print(180 * np.arccos(Inter_class_dispersion)/math.pi)

(5000, 2048)
(10, 1, 2048)
94.77689178280772


In [161]:
xc = [fl_features_train[ssl_labels_train == i] for i in np.unique(ssl_labels_train)]

print(xc[0].shape)

all_embedding = []

for x in xc: 
    
    cla_embedding = np.mean(x, axis=0, keepdims=True)
#     print(cla_embedding.shape)
    all_embedding.append(cla_embedding)
all_embedding = np.array(all_embedding)

print(all_embedding.shape)

Inter_class_dispersion = 1 - distance.cosine(all_embedding[i], all_embedding[j])
print(180 * np.arccos(Inter_class_dispersion)/math.pi)

(5000, 4096)
(10, 1, 4096)
95.03753673390938


In [62]:
xc = [fl_features_train[ssl_labels_train == i] for i in np.unique(ssl_labels_train)]

print(xc[0].shape)

all_embedding = []
for x in xc: 
    
    cla_embedding = np.mean(x, axis=0, keepdims=True)
#     print(cla_embedding.shape)
    all_embedding.append(cla_embedding)
all_embedding = np.array(all_embedding)

print(all_embedding.shape)

k = 0 
total = 0
for i in range(10):
    for j in range(i+1, 10):
        class_dispersion = 1 - distance.cosine(all_embedding[i], all_embedding[j])
        total  = class_dispersion + total
        k += 1
        Inter_class_dispersion = total/k
print(Inter_class_dispersion)
        
        

(5000, 4096)
(10, 1, 4096)
-0.11107864141878154


In [80]:
#  calcualte compactness
import math


xc = [ssl_features_train[ssl_labels_train == i] for i in np.unique(ssl_labels_train)]

o_compact = 0 

c = 0
for x in xc:
    all_compact = 0
    print(x.shape)
    for i in range(x.shape[0]):
        all_compact = all_compact + 1 - distance.cosine(x[i], all_embedding[c])
    o_compact = o_compact + all_compact
    print("class"+str(c)+"'s compactness is:", 180 * np.arccos(all_compact/x.shape[0])/math.pi)
    c += 1
    
print("overall compactness is:", 180 * np.arccos(o_compact/50000)/math.pi)

(5000, 2048)
class0's compactness is: 71.32903571038881
(5000, 2048)
class1's compactness is: 67.6464673874927
(5000, 2048)
class2's compactness is: 73.71511553986119
(5000, 2048)
class3's compactness is: 74.06747412292219
(5000, 2048)
class4's compactness is: 71.51522838329299
(5000, 2048)
class5's compactness is: 73.41277862007108
(5000, 2048)
class6's compactness is: 67.82569751759843
(5000, 2048)
class7's compactness is: 70.8060637814999
(5000, 2048)
class8's compactness is: 69.24716820519127
(5000, 2048)
class9's compactness is: 67.80259259486972
overall compactness is: 70.75412012129294


In [82]:
#  calcualte compactness



xc = [sp_features_train[sp_labels_train == i] for i in np.unique(sp_labels_train)]

o_compact = 0 

c = 0
for x in xc:
    all_compact = 0
    print(x.shape)
    for i in range(x.shape[0]):
        all_compact = all_compact + 1 - distance.cosine(x[i], all_embedding[c])
    o_compact = o_compact + all_compact
    print("class"+str(c)+"'s compactness is:", 180 * np.arccos(all_compact/x.shape[0])/math.pi)
    c += 1
    
print("overall compactness is:", 180 * np.arccos(o_compact/50000)/math.pi)




(5000, 2048)
class0's compactness is: 25.2915789053564
(5000, 2048)
class1's compactness is: 27.19969177887323
(5000, 2048)
class2's compactness is: 25.527930453032322
(5000, 2048)
class3's compactness is: 22.88985783535919
(5000, 2048)
class4's compactness is: 24.57593021693201
(5000, 2048)
class5's compactness is: 24.01253031147492
(5000, 2048)
class6's compactness is: 22.79114947884434
(5000, 2048)
class7's compactness is: 23.571704974994784
(5000, 2048)
class8's compactness is: 21.786599811294757
(5000, 2048)
class9's compactness is: 24.417771183913388
overall compactness is: 24.249271899576843


In [84]:
#  calcualte compactness



xc = [fl_features_train[sp_labels_train == i] for i in np.unique(sp_labels_train)]

o_compact = 0 

c = 0
for x in xc:
    all_compact = 0
    print(x.shape)
    for i in range(x.shape[0]):
        all_compact = all_compact + 1 - distance.cosine(x[i], all_embedding[c])
    o_compact = o_compact + all_compact
    print("class"+str(c)+"'s compactness is:", 180 * np.arccos(all_compact/x.shape[0])/math.pi)
    c += 1
    
print("overall compactness is:", 180 * np.arccos(o_compact/50000)/math.pi)

(5000, 4096)
class0's compactness is: 47.295589472045876
(5000, 4096)
class1's compactness is: 46.84459866791529
(5000, 4096)
class2's compactness is: 48.07159076901955
(5000, 4096)
class3's compactness is: 47.17617243038252
(5000, 4096)
class4's compactness is: 47.077012673371335
(5000, 4096)
class5's compactness is: 47.408549943006925
(5000, 4096)
class6's compactness is: 45.219147071832175
(5000, 4096)
class7's compactness is: 46.48448453978682
(5000, 4096)
class8's compactness is: 45.3517262123191
(5000, 4096)
class9's compactness is: 45.80945102193846
overall compactness is: 46.68040546791253
